In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import cv2
import numpy as np
from utils import *
from frame import extract,match,Frame


In [22]:
W,H=640,480
K=np.array([[1,0,W//2],[0,1,H//2],[0,0,1]])

In [23]:
frames=[]

In [24]:

def process_img(img,depth):

    frame=Frame(img,depth,K)
    frames.append(frame)


    if len(frames)<=1:
        return
    f_c=frames[-1] #current frame
    f_p=frames[-2] # previous frame
    ret,pose=match(f_p,f_c)

    # pose relative transformation of current frame  
    # with respect to previous frame 

    if ret is None:
        return
    
    if pose is None:
        return


    for kp1,kp2 in ret:
        u_p,v_p,_=denormalize(kp1,f_p.K)
        u_c,v_c,_=denormalize(kp2,f_c.K)

        cv2.circle(img,(u_p,v_p),color=(0,255,0),radius=3)
        cv2.line(img,(u_p,v_p),(u_c,v_c),color=(255,0,0))
    disp(img,"RGB")




In [25]:
if __name__ == "__main__":
    
    depth_paths='../dataset/rgbd_dataset_freiburg1_xyz/depth.txt'
    dlist=data(depth_paths)

    rgb_paths='../dataset/rgbd_dataset_freiburg1_xyz/rgb.txt'
    ilist=data(rgb_paths)

    dataset_path='../dataset/rgbd_dataset_freiburg1_xyz/'
    

    for i in range(len(dlist)):

        frame=cv2.imread(dataset_path+ilist[i])
        depth=cv2.imread(dataset_path+dlist[i],0)
        # print(frame.shape,depth.shape)
        W=frame.shape[1]
        H=frame.shape[0]
        # print(W,H)
        process_img(frame,depth)
        # print(frame)

        if frame is None:
            print("End of frame")
            break

        
        disp(depth,"Depth")
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()


        
